# Sistemas de Big Data - Apartado 2

Vamos a trabajar con el dataset de los resultados de todos los partidos de fútbol disputados entre selecciones nacionales desde 1872 hasta la actualidad, que ya utilizamos en la tarea de la entrega 3 de Big Data Aplicado (Hive). Puedes encontrar los datos en Kaggle, donde también podrás ver todos los detalles.

De los cuatro archivos que contiene el dataset, solo nos interesan dos:
    
  •	**results.csv:** que contiene la información de todos los partidos disputados, incluyendo los equipos, el marcador, el campeonato y la sede.

  •	**goalscorers.csv:** que contiene la información de todos los goles marcados en esos partidos. Para cada gol se indica el partido (fecha y equipos), el equipo y jugador que marca el gol, el minuto y dos indicadores que muestran si fue en propia puerta o de penalti.

In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, desc, sum, when, row_number, lit
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Football Analysis").getOrCreate()

results_url = "https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/results.csv"
goalscorers_url = "https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/goalscorers.csv"

!wget -O results.csv {results_url}
!wget -O goalscorers.csv {goalscorers_url}

results_df = spark.read.csv("results.csv", header=True, inferSchema=True)
goalscorers_df = spark.read.csv("goalscorers.csv", header=True, inferSchema=True)

--2025-04-26 10:57:02--  https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3628614 (3.5M) [text/plain]
Saving to: ‘results.csv’

results.csv         100%[===================>]   3.46M  10.6MB/s    in 0.3s    

2025-04-26 10:57:04 (10.6 MB/s) - ‘results.csv’ saved [3628614/3628614]

--2025-04-26 10:57:04--  https://raw.githubusercontent.com/tnavarrete-iedib/bigdata-24-25/refs/heads/main/goalscorers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK


Como analistas de datos, ahora nos piden nuevamente las mismas preguntas que en la tarea de Hive. Debes escribir un cuaderno de Google Colab que, utilizando PySpark y la librería Spark SQL, responda a cada una de las preguntas. Y debes responder a cada pregunta de dos maneras diferentes: utilizando métodos de DataFrame y ejecutando una sentencia SQL.

## 1 - Número de goles que ha marcado Lionel Messi (sin contar autogoles).

In [3]:
from pyspark.sql.functions import lower, when, col

goalscorers_df = goalscorers_df.withColumn("own_goal",
                                        when(lower(col("own_goal")) == "true", True)
                                        .otherwise(False))

goalscorers_df = goalscorers_df.withColumn("penalty",
                                        when(lower(col("penalty")) == "true", True)
                                        .otherwise(False))

results_df.createOrReplaceTempView("results")
goalscorers_df.createOrReplaceTempView("goalscorers")

print("Vistazo rápido de los datos 'results_df'':")
results_df.show(5)

print("\nVistazo rápido de los datos 'goalscorers_df':")
goalscorers_df.show(5)

# Usando DataFrame
messi_goals_df = goalscorers_df.filter((col("scorer") == "Lionel Messi") & (col("own_goal") == False)).count()

print("\n1. Número de goles que ha marcado Messi (sin contar autogoles):")
print(f"DataFrame: {messi_goals_df}")

# Sentencia SQL
messi_goals_sql = spark.sql("""
    SELECT COUNT(*) as goles_marcados
    FROM goalscorers
    WHERE scorer = 'Lionel Messi' AND own_goal = false
""")

print("SQL: ")
messi_goals_sql.show()


Vistazo rápido de los datos 'results_df'':
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  false|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
only showing top 5 rows


Vistazo rápido de los datos 'goalscorers_df':
+----------+---------+---------+---------+----------------+------+

## 2 - Listado de los 5 partidos más recientes que ha jugado la selección española

In [4]:
# Usando DataFrame
spain_matches_df = results_df.filter((col("home_team") == "Spain") | (col("away_team") == "Spain")) \
                           .orderBy(col("date").desc()) \
                           .limit(5)

print("DataFrame:")
spain_matches_df.show(5)

# Sentencia SQL
spain_matches_sql = spark.sql("""
    SELECT *
    FROM results
    WHERE home_team = 'Spain' OR away_team = 'Spain'
    ORDER BY date DESC
    LIMIT 5
""")

print("SQL:")
spain_matches_sql.show(5)

DataFrame:
+----------+-----------+-----------+----------+----------+-------------------+--------------------+-----------+-------+
|      date|  home_team|  away_team|home_score|away_score|         tournament|                city|    country|neutral|
+----------+-----------+-----------+----------+----------+-------------------+--------------------+-----------+-------+
|2025-03-23|      Spain|Netherlands|         3|         3|UEFA Nations League|            Valencia|      Spain|  false|
|2025-03-20|Netherlands|      Spain|         2|         2|UEFA Nations League|           Rotterdam|Netherlands|  false|
|2024-11-18|      Spain|Switzerland|         3|         2|UEFA Nations League|Santa Cruz de Ten...|      Spain|  false|
|2024-11-15|    Denmark|      Spain|         1|         2|UEFA Nations League|          Copenhagen|    Denmark|  false|
|2024-10-15|      Spain|     Serbia|         3|         0|UEFA Nations League|             Cordoba|      Spain|  false|
+----------+-----------+-----

## 3 - Número de goles que ha marcado España en toda su historia (desde results)

In [6]:
# Usando DataFrame
spain_total_goals_df = results_df.filter((col("home_team") == "Spain") | (col("away_team") == "Spain")) \
                               .withColumn("spain_goals",
                                          when(col("home_team") == "Spain", col("home_score"))
                                          .otherwise(col("away_score"))) \
                               .agg(sum("spain_goals").alias("goles_totales"))

print("DataFrame:")
spain_total_goals_df.show()

# Sentencia SQL
spain_total_goals_sql = spark.sql("""
    SELECT SUM(
        CASE
            WHEN home_team = 'Spain' THEN home_score
            ELSE away_score
        END
    ) as goles_totales
    FROM results
    WHERE home_team = 'Spain' OR away_team = 'Spain'
""")

print("SQL:")
spain_total_goals_sql.show()

DataFrame:
+-------------+
|goles_totales|
+-------------+
|         1567|
+-------------+

SQL:
+-------------+
|goles_totales|
+-------------+
|         1567|
+-------------+



## 4 - Listado de los 5 máximos goleadores con la selección española (sin contar autogoles)

In [8]:
# Usando DataFrame
top_spanish_scorers_df = goalscorers_df.filter((col("team") == "Spain") & (col("own_goal") == False)) \
                                     .groupBy("scorer") \
                                     .count() \
                                     .orderBy(col("count").desc()) \
                                     .limit(5)

print("DataFrame:")
top_spanish_scorers_df.show(5)

# Sentencia SQL
top_spanish_scorers_sql = spark.sql("""
    SELECT scorer, COUNT(*) as goles
    FROM goalscorers
    WHERE team = 'Spain' AND own_goal = false
    GROUP BY scorer
    ORDER BY goles DESC
    LIMIT 5
""")

print("SQL:")
top_spanish_scorers_sql.show(5)

DataFrame:
+---------------+-----+
|         scorer|count|
+---------------+-----+
|    David Villa|   41|
|           Raúl|   32|
|  Álvaro Morata|   29|
|Fernando Torres|   28|
|Fernando Hierro|   25|
+---------------+-----+

SQL:
+---------------+-----+
|         scorer|goles|
+---------------+-----+
|    David Villa|   41|
|           Raúl|   32|
|  Álvaro Morata|   29|
|Fernando Torres|   28|
|Fernando Hierro|   25|
+---------------+-----+



## 5 - Listado de los jugadores españoles que han marcado algún gol de penalti en alguna Eurocopa (UEFA Euro), ordenados alfabéticamente

In [9]:
# Usando DataFrame
spanish_euro_penalty_scorers_df = goalscorers_df.join(
    results_df,
    (goalscorers_df["date"] == results_df["date"]) &
    (((goalscorers_df["home_team"] == results_df["home_team"]) & (goalscorers_df["away_team"] == results_df["away_team"])) |
     ((goalscorers_df["home_team"] == results_df["away_team"]) & (goalscorers_df["away_team"] == results_df["home_team"]))),
    "inner"
) \
.filter((col("team") == "Spain") &
        (col("tournament").like("%UEFA Euro%")) &
        (col("penalty") == True)) \
.select("scorer") \
.distinct() \
.orderBy("scorer")

print("DataFrame:")
spanish_euro_penalty_scorers_df.show(100)

# Sentencia SQL
spanish_euro_penalty_scorers_sql = spark.sql("""
    SELECT DISTINCT g.scorer
    FROM goalscorers g
    JOIN results r
    ON g.date = r.date
    AND ((g.home_team = r.home_team AND g.away_team = r.away_team)
         OR (g.home_team = r.away_team AND g.away_team = r.home_team))
    WHERE g.team = 'Spain'
    AND r.tournament LIKE '%UEFA Euro%'
    AND g.penalty = true
    ORDER BY g.scorer
""")

print("SQL:")
spanish_euro_penalty_scorers_sql.show(100)

DataFrame:
+--------------------+
|              scorer|
+--------------------+
|      Andrés Iniesta|
|         Daniel Ruiz|
|         David Villa|
|     Fernando Hierro|
|Francisco José Ca...|
|     Gaizka Mendieta|
|      José Claramunt|
|  Juan Antonio Señor|
|              Míchel|
|               Pirri|
|        Sergio Ramos|
|         Xabi Alonso|
|       Álvaro Morata|
+--------------------+

SQL:
+--------------------+
|              scorer|
+--------------------+
|      Andrés Iniesta|
|         Daniel Ruiz|
|         David Villa|
|     Fernando Hierro|
|Francisco José Ca...|
|     Gaizka Mendieta|
|      José Claramunt|
|  Juan Antonio Señor|
|              Míchel|
|               Pirri|
|        Sergio Ramos|
|         Xabi Alonso|
|       Álvaro Morata|
+--------------------+



## 6 - Listado de los 5 máximos goleadores de las fases finales de los mundiales (FIFA World Cup) sin contar autogoles.

In [11]:
# Usando DataFrame
top_world_cup_scorers_df = goalscorers_df.join(
    results_df,
    (goalscorers_df["date"] == results_df["date"]) &
    (((goalscorers_df["home_team"] == results_df["home_team"]) & (goalscorers_df["away_team"] == results_df["away_team"])) |
     ((goalscorers_df["home_team"] == results_df["away_team"]) & (goalscorers_df["away_team"] == results_df["home_team"]))),
    "inner"
) \
.filter((col("tournament").like("%FIFA World Cup%")) &
        (col("own_goal") == False)) \
.groupBy("scorer") \
.count() \
.orderBy(col("count").desc()) \
.limit(5)

print("DataFrame:")
top_world_cup_scorers_df.show(5)

# Sentencia SQL
top_world_cup_scorers_sql = spark.sql("""
    SELECT g.scorer, COUNT(*) as goles
    FROM goalscorers g
    JOIN results r
    ON g.date = r.date
    AND ((g.home_team = r.home_team AND g.away_team = r.away_team)
         OR (g.home_team = r.away_team AND g.away_team = r.home_team))
    WHERE r.tournament LIKE '%FIFA World Cup%'
    AND g.own_goal = false
    GROUP BY g.scorer
    ORDER BY goles DESC
    LIMIT 5
""")

print("SQL:")
top_world_cup_scorers_sql.show(5)

DataFrame:
+-----------------+-----+
|           scorer|count|
+-----------------+-----+
|Cristiano Ronaldo|   44|
|     Lionel Messi|   41|
|      Carlos Ruiz|   39|
|      Luis Suárez|   36|
|         Ali Daei|   35|
+-----------------+-----+

SQL:
+-----------------+-----+
|           scorer|goles|
+-----------------+-----+
|Cristiano Ronaldo|   44|
|     Lionel Messi|   41|
|      Carlos Ruiz|   39|
|      Luis Suárez|   36|
|         Ali Daei|   35|
+-----------------+-----+

